# Declaring Variables

In [ ]:
import sys
sys.path
if "/home/jovyan/utils" not in sys.path:
    sys.path.append("/home/jovyan/utils")
if "/home/jovyan/tests/test_utils" not in sys.path:
    sys.path.append("/home/jovyan/tests/test_utils")

from importlib import reload
import json_parsing
reload(json_parsing)
import custom_reporter
import scrapbook as sb

report_name = "CompositeUnitTestNotebook"   # .html report name

In [ ]:
result_json_list = [] # Variable for displaying information about tests in .json format
report = custom_reporter.Report('Check_Name', 'Status', 'Description', 'Error_Description', 'Time_Execution', report_name=report_name)

In [ ]:
def add_result_to_custom_report(result_list: list, description: str, check_name: str, result_json_list: str="") -> dict:
    #delete all empty elements from result_list
    result_list = [item for item in result_list if item != '']
    
    #delete last element from result_list if list have extra logs (not status)
    while result_list:
        status = result_list[-1]
        if status.startswith('OK') or status.startswith('FAILED'):
            break
        else:
            result_list.pop()

    status = result_list[-1]
    time_exec = result_list[-2]
    error_description = "-"
    if status == "OK":
        status = "Passed"
    elif status.startswith("FAILED"):
        status = "Failed"
        error_description = result_list[-4]

    #add info to report
    report.append("Status", status,
                  Check_Name = check_name,
                  Description = description,
                  Error_Description = error_description,
                  Time_Execution = time_exec)

    # create variable in .json format to Fortnight test
    json_report = json_parsing.JsonReport()
    if result_json_list == "":
        result_json_list = json_report.add_check(check_name, status, description, error_description, time_exec)
    else:
        result_json_list = json_report.add_check_with_existing_json(result_json_list, check_name, status, description, 
                                                                    error_description, time_exec)
    json_data = {
        "overall_result": json_report.overall_result,
        "checks": result_json_list
    }
    print(f"json_data={json_data}")
    
    return json_data

# Test cases
## #2 Checks the creation of .html

In [ ]:
result = !python /home/jovyan/tests/unittests/reports/html_creation_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh run for notebook with '--html=true' flag enabled", 
                            "Checks the creation of .html",
                            result_json_list)

## #3 Checks the creation of .pdf

In [ ]:
result = !python /home/jovyan/tests/unittests/reports/pdf_creation_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh run for notebook without flags enabled", 
                            "Checks the creation of .pdf",
                            result_json_list)

## #4 Checks the disabling creation of .pdf

In [ ]:
result = !python /home/jovyan/tests/unittests/reports/pdf_disabling_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh shell for notebook with '--pdf=false' flag enabled", 
                            "Checks the disabling creation of .pdf",
                            result_json_list)

## #5 Checks composite run

In [ ]:
result = !python /home/jovyan/tests/unittests/shells/bulk_call_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh shell launch using a composite .yaml file", 
                            "Checks composite run",
                            result_json_list)

## #6 Checks '-j' shell flag

In [ ]:
result = !python /home/jovyan/tests/unittests/shells/j_flag_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh shell launch using a '-j' flag (launch via json)", 
                            "Checks '-j' shell flag",
                            result_json_list)

## #7 Checks '-o' shell flag

In [ ]:
result = !python /home/jovyan/tests/unittests/shells/o_flag_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh shell launch using '-o' flag for creating reports in additional folder", 
                            "Checks '-o' shell flag",
                            result_json_list)

## #8 Checks '-y' shell flag

In [ ]:
result = !python /home/jovyan/tests/unittests/shells/y_flag_test.py
result_json_list = add_result_to_custom_report(result, 
                            "Makes run.sh shell launch using a '-y' flag (launch via yaml)", 
                            "Checks '-y' shell flag",
                            result_json_list)

## #9 Checks integration with S3

In [ ]:
#result = !python /home/jovyan/tests/unittests/integrations/s3_integration_test.py
#result_json_list = add_result_to_custom_report(result,
#                                               "Checking the functionality of integration with S3. run.sh is used to check the s3.uploadReportsByExecutedNotebookPath method",
#                                               "Checks integration with S3",
#                                               result_json_list)

In [ ]:
json_parsing.JsonReport.save_to_file(result_json_list, "/home/jovyan/out/tests", f"{report_name}.json")
sb.glue("report", report.dict())

# Final check

In [ ]:
result_json_list['overall_result'] == 'Passed'